In [1]:
from autogen.agentchat import AssistantAgent, UserProxyAgent
from autogen.oai.openai_utils import config_list_from_json
from types import SimpleNamespace
import requests

In [ ]:
class CustomModelClient:
    def __init__(self, config, **kwargs):
        print(f"CustomModelClient config: {config}")
        self.api_key = config.get("api_key")
        self.api_url = "http://localhost:8000/generate"
        self.model = config.get("model", "gpt-3.5-turbo")
        self.max_tokens = config.get("max_tokens", 4000)
        self.temperature = config.get("temperature", 0.8)
        self.top_p = config.get("top_p", 1)
        self.presence_penalty = config.get("presence_penalty", 1)

        print(f"Initialized CustomModelClient with model {self.model}")

    def create(self, params):

        messages = params.get("messages", [])
        if isinstance(messages, list):
            prompt = "\n".join([m.get("content", "") for m in messages if m.get("role") in ("user", "system")])
        else:
            prompt = str(messages)

        data = {
            "max_tokens": self.max_tokens if self.max_tokens else 4000,
            "temperature": self.temperature,
            "top_p": self.top_p if self.top_p else 0.9,
            "prompt": prompt,
        }

        response = requests.post(self.api_url, json=data)
        response.raise_for_status()  # Raise an exception for HTTP errors

        api_response = response.json()
        print(api_response)
        # Convert API response to SimpleNamespace for compatibility
        client_response = SimpleNamespace()
        client_response.choices = []
        client_response.model = self.model

        for choice in api_response.get("choices", []):
            client_choice = SimpleNamespace()
            client_choice.message = SimpleNamespace()
            client_choice.message.content = choice.get("message", {}).get("content")
            client_choice.message.function_call = None
            client_response.choices.append(client_choice)

        return client_response

    def message_retrieval(self, response):
        """Retrieve the messages from the response."""
        choices = response.choices
        return [choice.message.content for choice in choices]

    def cost(self, response) -> float:
        """Calculate the cost of the response."""
        # Implement cost calculation if available from your API
        response.cost = 0
        return 0

    @staticmethod
    def get_usage(response):
        # Implement usage tracking if available from your API
        return {}


In [3]:
config_list_custom = config_list_from_json(
    "OAI_CONFIG_LIST.json",
    filter_dict={"model_client_cls": ["CustomModelClient"]},
)

In [4]:
config_list_custom

[{'model': 'Qwen/Qwen3-0.6B', 'model_client_cls': 'CustomModelClient'}]

In [ ]:
assistant = AssistantAgent(name="assistant", llm_config={"config_list": config_list_custom})
user_proxy = UserProxyAgent(
    "user_proxy",
    code_execution_config={
        #"work_dir": "coding",
        "use_docker": False,
    },
)

assistant.register_model_client(model_client_cls=CustomModelClient)

[autogen.oai.client: 05-20 11:08:45] {871} INFO - Detected custom model client in config: CustomModelClient, model client can not be used until register_model_client is called.
CustomModelClient config: OpenAILLMConfigEntry(api_type='openai', model='Qwen/Qwen3-0.6B', model_client_cls='CustomModelClient', tags=[])
Initialized CustomModelClient with model Qwen/Qwen3-0.6B


In [7]:
user_proxy.initiate_chat(
    assistant,
    message="What is the tallest building in the world?",
)

user_proxy (to assistant):

What is the tallest building in the world?

--------------------------------------------------------------------------------
{'choices': [{'message': {'content': 'The tallest building in the world is the Burj Khalifa in Dubai, UAE. This building stands at 828 meters and is the tallest in the world. \n\nTERMINATE'}}]}
assistant (to user_proxy):

The tallest building in the world is the Burj Khalifa in Dubai, UAE. This building stands at 828 meters and is the tallest in the world. 

TERMINATE

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (857adda9-40d0-45c9-a671-5b85e2d0ec27): User requested to end the conversation


ChatResult(chat_id=None, chat_history=[{'content': 'What is the tallest building in the world?', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'The tallest building in the world is the Burj Khalifa in Dubai, UAE. This building stands at 828 meters and is the tallest in the world. \n\nTERMINATE', 'role': 'user', 'name': 'assistant'}], summary='The tallest building in the world is the Burj Khalifa in Dubai, UAE. This building stands at 828 meters and is the tallest in the world. \n\n', cost={'usage_including_cached_inference': {'total_cost': 0}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=['exit'])